# GWO DailyまたはHoulyの複数測点csvを測点別csvに分割する
**Author: Jun Sasaki**  **Coded on Sep. 16, 2018, revised on January 10, 2022**<br>
気象データベース地上観測（GWO）DVD時別値は，1961年から1990年までは3時間間隔データとなっている．GWOのDVDから1地点1ファイルとして複数年一括出力したものを年毎に分ける．1991年以降は1時間間隔となっている．本コードはどちらにも，あるいは混在していても適用可能である．<br>
このほかに日別値にも同様に対応した．<br>
【解説】GWOのDVDでは全年全観測点のデータを一つのcsvファイルとして出力できる．これを観測点別のファイルに分割するのが目的．<br>
#### 注意
* 入力csvファイルのエンコードはSHIFT-JIS，CRLF，分割後の観測点別ファイルのエンコードはSHIFT-JIS，CRLFである．
* データベースはSQLViewer7を立ち上げ，「有効データ」および「閾値ソート」のチェックを外し，「全データベース項目」にチェックを入れ，全観測所を選び，全期間を指定して，CSV出力として実行する．
* 時別値は1日の最後のデータ時刻が24時のため，年末24時のデータは翌年年初0時のデータを意味する．

In [ ]:
import pandas as pd
import os
import sys

## stn_dictを作成しておく
測点名（漢字）を自動認識し，その測点名に対応するディレクトリ名をstn_dictから得る

In [ ]:
stn_dict={"稚内":"Wakkanai", "北見枝幸":"Kitamiesashi", "羽幌":"Haboro", "雄武":"Oumu", "留萌":"Rumoi", "旭川":"Asahikawa", \
          "網走":"Abashiri", "小樽":"Otaru", "札幌":"Sapporo", "岩見沢":"Iwamizawa", "帯広":"Obihiro", "釧路":"Kushiro", \
          "根室":"Nemuro", "寿都":"Suttu", "室蘭":"Muroran", "苫小牧":"Tomakomai", \
          "浦河":"Urakawa", "江差":"Esashi", "函館":"Hakodate", "倶知安":"Kutchan", "紋別":"Monbetsu", "広尾":"Hiroo", \
          "大船渡":"Ofunato", "新庄":"Shinjo", "若松":"Wakamatsu", "深浦":"Fukaura", \
          "青森":"Aomori", "むつ":"Mutsu", "八戸":"Hachinohe", "秋田":"Akita", "盛岡":"Morioka", "宮古":"Miyako", \
          "酒田":"Sakata", "山形":"Yamagata", "仙台":"Sendai", "石巻":"Ishinomaki", \
          "福島":"Fukushima", "白河":"Shirakawa", "小名浜":"Onahama", "輪島":"Wajima", "相川":"Aikawa", "新潟":"Niigata", \
          "金沢":"Kanazawa", "伏木":"Fushiki", "富山":"Toyama", "長野":"Nagano", \
          "高田":"Takada", "宇都宮":"Utsunomiya", "福井":"Fukui", "高山":"Takayama", "松本":"Matsumoto", \
          "諏訪":"Suwa", "軽井沢":"Karuizawa", "前橋":"Maebashi", "熊谷":"Kumagaya", "水戸":"Mito", \
          "敦賀":"Tsuruga", "岐阜":"Gifu", "名古屋":"Nagoya", "飯田":"Iida", "甲府":"Kofu", \
          "河口湖":"Kawaguchiko", "秩父":"Chichibu", "館野":"Tateno", "銚子":"Choshi", "上野":"Ueno", \
          "津":"Tsu", "伊良湖":"Irago", "浜松":"Hamamatsu", "御前崎":"Omaezaki", "静岡":"Shizuoka", \
          "三島":"Mishima", "東京":"Tokyo", "尾鷲":"Owase", "石廊崎":"Irozaki", "網代":"Ajiro", \
          "横浜":"Yokohama", "館山":"Tateyama", "勝浦":"Katsuura", "大島":"Oshima", "三宅島":"Miyakejima", \
          "八丈島":"Hachijojima", "千葉":"Chiba", "四日市":"Yokkaichi", "日光":"Nikko", "西郷":"Saigo", \
          "松江":"Matsue", "境":"Sakai", "米子":"Yonago", "鳥取":"Tottori", "豊岡":"Toyooka", "舞鶴":"Maiduru", \
          "伊吹山":"Ibukiyama", "萩":"Hagi", "浜田":"Hamada", "津山":"Tsuyama", \
          "京都":"Kyoto", "彦根":"Hikone", "下関":"Shimonoseki", "広島":"Hiroshima", "呉":"Kure", \
          "福山":"Fukuyama", "岡山":"Okayama", "姫路":"Himeji", "神戸":"Kobe", "大阪":"Osaka", \
          "洲本":"Sumoto", "和歌山":"Wakayama", "潮岬":"Shionomisaki", "奈良":"Nara", "山口":"Yamaguchi", \
          "厳原":"Izuhara", "平戸":"Hirado", "福岡":"Fukuoka", "飯塚":"Iiduka", "佐世保":"Sasebo", \
          "佐賀":"Saga", "日田":"Hita", "大分":"Oita", "長崎":"Nagasaki", "熊本":"Kumamoto", \
          "阿蘇山":"Asosan", "延岡":"Nobeoka", "阿久根":"Akune", "人吉":"Hitoyoshi", "鹿児島":"Kagoshima", \
          "都城":"Miyakonojo", "宮崎":"Miyazaki", "枕崎":"Makurazaki", "油津":"Aburatsu", "屋久島":"Yakushima", \
          "種子島":"Tanegashima", "牛深":"Ushibuka", "福江":"Fukue", "松山":"Matsuyama", "多度津":"Tadotsu", \
          "高松":"Takamatsu", "宇和島":"Uwajima", "高知":"Kochi", "剣山":"Tsurugisan", "徳島":"Tokushima", \
          "宿毛":"Sukumo", "清水":"Shimizu", "室戸岬":"Murotomisaki", "名瀬":"Nase", "与那国島":"Yonakunijima", \
          "石垣島":"Ishigakijima", "宮古島":"Miyakojima", "久米島":"Kumejima", "那覇":"Naha", "名護":"Nago", \
          "沖永良部":"Okinoerabu", "南大東島":"Minamidaitojima", "父島":"Chichijima", "南鳥島":"Minamitorishima"}

In [ ]:
def GWO_stns2stn(fname="Matsue-Sakai-Yonago-Tottori", dir="../GWO/Daily/", hourly=False, overwrite=False):
    '''Divide a file containing multiple stations csv to each station csv'''

    if hourly:
        names = ["KanID","Kname","KanID_1","YYYY","MM","DD","HH","lhpa","lhpaRMK","shpa","shpaRMK","kion","kionRMK","stem",\
                 "stemRMK","rhum","rhumRMK","muki","mukiRMK","sped","spedRMK","clod","clodRMK","tnki","tnkiRMK","humd","humdRMK",\
                 "lght","lghtRMK","slht","slhtRMK","kous","kousRMK"]
    else:
        names = ["KanID","Kname","KanID_1","YYYY","MM","DD","avrLhpa","avrLhpaRMK","avrShpa","avrShpaRMK","minShpa","minShpaRMK",\
                 "avrKion","avrKionRMK","maxKion","maxKionRMK","minKion","minKionRMK","avrStem","avrStemRMK","avrRhum","avrRhumRMK",\
                  "minRhum","minRhumRMK","avrSped","avrSpedRMK","maxSped","maxSpedRMK","maxMuki","maxMukiRMK","maxSSpd","maxSSpdRMK",\
                  "maxSMuk","maxSMukRMK","avrClod","avrClodRMK","daylght","daylghtRMK","sunlght","sunlghtRMK","amtEva","amtEvaRMK",\
                  "dayPrec","dayPrecRMK","maxHPrc","maxHPrcRMK","maxMPrc","maxMPrcRMK","talSnow","talSnowRMK","daySnow","daySnowRMK",\
                  "tenki1","tenki1RMK","tenki2","tenki2RMK","apCode1","apCode2","apCode3","apCode4","apCode5","strgTim","strgTimRMK"]
    
    fpath=dir + fname + ".csv"
    print("Reading ", fpath)
    df = pd.read_csv(fpath, header=None, names=names, dtype="str", encoding="SHIFT-JIS")
    for stn in set(df['Kname']):
        print(stn)
        df_stn = df[df['Kname']==stn]
        dirpath_stn = dir + stn_dict[stn]
        if hourly:
            fpath_stn = dirpath_stn + "/" + stn_dict[stn] + fname[-9:] + ".csv"
        else:
            fpath_stn = dirpath_stn + "/" + stn_dict[stn] + "1961-2017" + ".csv"  ### output CSV file path for each year
        if not os.path.isdir(dirpath_stn):  ### directoryが存在しない場合は作成
            print("Creating directory of " + dirpath_stn)
            os.mkdir(dirpath_stn)
        if not os.path.isfile(fpath_stn):
            print("Creating " + fpath_stn)
            df_stn.to_csv(fpath_stn, header=None, index=False, encoding="SHIFT-JIS")  ### エンコードをSHIFT-JISとする
        elif overwrite:
            print("Overwriting " + fpath_stn)
            df_stn.to_csv(fpath_stn, header=None, index=False, encoding="SHIFT-JIS")  ### エンコードをSHIFT-JISとする
        else:
            print("Already existing " + fpath_stn)

In [ ]:
dirpath = "d:\dat\GWO\"
end_year = "2020"

### GWO Daily data
アーカイブデータファイル名を与え，観測点別のファイルに分割する．観測点名はファイル中の漢字名からstn_dictによって自動設定される．

In [ ]:
GWO_stns2stn(fname="GWODaily1961-{}".format(end_year), dir="{}Daily/".format(dirpath), hourly=False, overwrite=False)

### GWO Hourly data
1961-1990のGWOHourly1961-1990.csvと1991-2017のGWOHourly1991-2017.csvをそれぞれ，観測点毎のファイルに分割する

In [ ]:
GWO_stns2stn(fname="GWOHourly1961-1990", dir="{}Hourly/".format(dirpath), hourly=True, overwrite=False)

In [ ]:
GWO_stns2stn(fname="GWOHourly1991-{}".format(end_year), dir="{}Hourly/".format(dirpath), hourly=True, overwrite=False)